In [ ]:
import pandas as pd
import numpy as np

# Extracting raw data (the fact-like table )
#ps:the folder consists of 5 files :Accounts,subscriptions,support_tickets,feature_usage,churn_events,but for brevity we will show only subscriptions and accounts here
subs_raw = pd.read_csv("data_raw/subscriptions.csv")
print("Raw shape:", subs_raw.shape)
subs_raw.head()

Raw shape: (5000, 14)


,subscription_id,account_id,start_date,end_date,plan_tier,seats,mrr_amount,arr_amount,is_trial,upgrade_flag,downgrade_flag,churn_flag,billing_frequency,auto_renew_flag
0,S-8cec59,A-3c1a3f,2023-12-23,2024-04-12,Enterprise,14,2786,33432,False,False,False,True,monthly,True
1,S-0f6f44,A-9b9fe9,2024-06-11,NaN,Pro,17,833,9996,False,False,False,False,monthly,True
2,S-51c0d1,A-659280,2024-11-25,NaN,Enterprise,62,0,0,True,True,False,False,annual,False
3,S-f81687,A-e7a1e2,2024-11-23,2024-12-13,Enterprise,5,995,11940,False,False,False,True,monthly,True
4,S-cff5a2,A-ba6516,2024-01-10,NaN,Enterprise,27,5373,64476,False,False,False,False,monthly,True


In [2]:
# structure and missing values
subs_raw.info()
print("\nMissing values per column:")
print(subs_raw.isna().sum())

print("\nColumns:")
print(list(subs_raw.columns))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   subscription_id    5000 non-null   object
 1   account_id         5000 non-null   object
 2   start_date         5000 non-null   object
 3   end_date           486 non-null    object
 4   plan_tier          5000 non-null   object
 5   seats              5000 non-null   int64 
 6   mrr_amount         5000 non-null   int64 
 7   arr_amount         5000 non-null   int64 
 8   is_trial           5000 non-null   bool  
 9   upgrade_flag       5000 non-null   bool  
 10  downgrade_flag     5000 non-null   bool  
 11  churn_flag         5000 non-null   bool  
 12  billing_frequency  5000 non-null   object
 13  auto_renew_flag    5000 non-null   bool  
dtypes: bool(5), int64(3), object(6)
memory usage: 376.1+ KB

Missing values per column:
subscription_id         0
account_id              0


In [3]:
# CLEAN & TRANSFORM subscriptions
 #Here we used a copy of the raw data to preserve original data
subs_clean = subs_raw.copy()

# 1) Convert date columns to datetime
subs_clean["start_date"] = pd.to_datetime(subs_clean["start_date"], errors="coerce")
subs_clean["end_date"] = pd.to_datetime(subs_clean["end_date"], errors="coerce")

# 2) Recompute churn_flag just to be safe (1 = churned, 0 = active)
subs_clean["churn_flag"] = subs_clean["end_date"].notna().astype(int)

# 3) Compute tenure in days (if no end_date, use a reference 'today' date)
reference_date = pd.Timestamp("2025-12-01")
end_or_today = subs_clean["end_date"].fillna(reference_date)
subs_clean["tenure_days"] = (end_or_today - subs_clean["start_date"]).dt.days

# Quick check
subs_clean[["subscription_id", "start_date", "end_date", "churn_flag", "tenure_days"]].head()
subs_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   subscription_id    5000 non-null   object        
 1   account_id         5000 non-null   object        
 2   start_date         5000 non-null   datetime64[ns]
 3   end_date           486 non-null    datetime64[ns]
 4   plan_tier          5000 non-null   object        
 5   seats              5000 non-null   int64         
 6   mrr_amount         5000 non-null   int64         
 7   arr_amount         5000 non-null   int64         
 8   is_trial           5000 non-null   bool          
 9   upgrade_flag       5000 non-null   bool          
 10  downgrade_flag     5000 non-null   bool          
 11  churn_flag         5000 non-null   int64         
 12  billing_frequency  5000 non-null   object        
 13  auto_renew_flag    5000 non-null   bool          
 14  tenure_d

In [4]:
# LOAD: Export clean subscriptions
import os
os.makedirs("data_cleaned", exist_ok=True)

subs_clean.to_csv("data_cleaned/fact_subscriptions_clean.csv", index=False)
print(" Saved data cleaned/fact_subscriptions_clean.csv")
print(f"   {subs_clean.shape[0]} rows, {subs_clean.shape[1]} columns")


 Saved data cleaned/fact_subscriptions_clean.csv
   5000 rows, 15 columns


In [6]:
#  cleaning accounts.csv (DimAccounts)

accounts_raw = pd.read_csv("data_raw/accounts.csv")
print("Accounts raw shape:", accounts_raw.shape)
accounts_raw.head()


Accounts raw shape: (500, 10)


,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag
0,A-2e4581,Company_0,EdTech,US,2024-10-16,partner,Basic,9,False,False
1,A-43a9e3,Company_1,FinTech,IN,2023-08-17,other,Basic,18,False,True
2,A-0a282f,Company_2,DevTools,US,2024-08-27,organic,Basic,1,False,False
3,A-1f0ac7,Company_3,HealthTech,UK,2023-08-27,other,Basic,24,True,False
4,A-ce550d,Company_4,HealthTech,US,2024-10-27,event,Enterprise,35,False,True


In [ ]:
accounts_raw.info()
print("\nMissing values per column:")
print(accounts_raw.isna().sum())
#after running,we found no issues with this file


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   account_id       500 non-null    object
 1   account_name     500 non-null    object
 2   industry         500 non-null    object
 3   country          500 non-null    object
 4   signup_date      500 non-null    object
 5   referral_source  500 non-null    object
 6   plan_tier        500 non-null    object
 7   seats            500 non-null    int64 
 8   is_trial         500 non-null    bool  
 9   churn_flag       500 non-null    bool  
dtypes: bool(2), int64(1), object(7)
memory usage: 32.4+ KB

Missing values per column:
account_id         0
account_name       0
industry           0
country            0
signup_date        0
referral_source    0
plan_tier          0
seats              0
is_trial           0
churn_flag         0
dtype: int64


In [ ]:
accounts_clean = accounts_raw.copy()

# Convert signup_date to datetime if it exists
if "signup_date" in accounts_clean.columns:
    accounts_clean["signup_date"] = pd.to_datetime(accounts_clean["signup_date"], errors="coerce")
    #this has transformed all dates correctly,no missing values

# Fill missing categorical values with "Unknown"
for col in accounts_clean.select_dtypes(include="object").columns:
    accounts_clean[col] = accounts_clean[col].fillna("Unknown")

accounts_clean.info()
accounts_clean.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   account_id       500 non-null    object        
 1   account_name     500 non-null    object        
 2   industry         500 non-null    object        
 3   country          500 non-null    object        
 4   signup_date      500 non-null    datetime64[ns]
 5   referral_source  500 non-null    object        
 6   plan_tier        500 non-null    object        
 7   seats            500 non-null    int64         
 8   is_trial         500 non-null    bool          
 9   churn_flag       500 non-null    bool          
dtypes: bool(2), datetime64[ns](1), int64(1), object(6)
memory usage: 32.4+ KB


,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag
0,A-2e4581,Company_0,EdTech,US,2024-10-16,partner,Basic,9,False,False
1,A-43a9e3,Company_1,FinTech,IN,2023-08-17,other,Basic,18,False,True
2,A-0a282f,Company_2,DevTools,US,2024-08-27,organic,Basic,1,False,False
3,A-1f0ac7,Company_3,HealthTech,UK,2023-08-27,other,Basic,24,True,False
4,A-ce550d,Company_4,HealthTech,US,2024-10-27,event,Enterprise,35,False,True


In [10]:
accounts_clean.to_csv("data_cleaned/dim_accounts_clean.csv", index=False)
print(" Saved data_cleaned/dim_accounts_clean.csv")
print(f"   {accounts_clean.shape[0]} rows, {accounts_clean.shape[1]} columns")


 Saved data_cleaned/dim_accounts_clean.csv
   500 rows, 10 columns


In [ ]:
# load support_tickets.csv

tickets_raw = pd.read_csv("data_raw/support_tickets.csv")
print("Support tickets raw shape:", tickets_raw.shape)
tickets_raw.head()



Support tickets raw shape: (2000, 9)


,ticket_id,account_id,submitted_at,closed_at,resolution_time_hours,priority,first_response_time_minutes,satisfaction_score,escalation_flag
0,T-0024de,A-712f1c,2023-07-27,2023-07-28 03:00:00,27.0,high,74,NaN,False
1,T-4d04b9,A-e43bf7,2024-07-08,2024-07-09 03:00:00,27.0,urgent,144,NaN,False
2,T-d5e12f,A-0f3e88,2024-10-17,2024-10-17 19:00:00,19.0,urgent,93,4.0,False
3,T-dfce9a,A-4c56c9,2024-09-08,2024-09-09 23:00:00,47.0,medium,126,5.0,False
4,T-c59f77,A-6f8ad2,2024-11-30,2024-12-01 02:00:00,26.0,medium,8,NaN,False


In [12]:
tickets_raw.info()
print("\nMissing values per column:")
print(tickets_raw.isna().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ticket_id                    2000 non-null   object 
 1   account_id                   2000 non-null   object 
 2   submitted_at                 2000 non-null   object 
 3   closed_at                    2000 non-null   object 
 4   resolution_time_hours        2000 non-null   float64
 5   priority                     2000 non-null   object 
 6   first_response_time_minutes  2000 non-null   int64  
 7   satisfaction_score           1175 non-null   float64
 8   escalation_flag              2000 non-null   bool   
dtypes: bool(1), float64(2), int64(1), object(5)
memory usage: 127.1+ KB

Missing values per column:
ticket_id                        0
account_id                       0
submitted_at                     0
closed_at                        0
resolution_time

In [13]:
tickets_clean = tickets_raw.copy()

# Convert date columns to datetime (adjust names if different)
for col in tickets_clean.columns:
    if "date" in col.lower():
        tickets_clean[col] = pd.to_datetime(tickets_clean[col], errors="coerce")

# Drop rows with missing critical IDs
tickets_clean = tickets_clean.dropna(subset=[tickets_clean.columns[0]])  # first column assumed ID

tickets_clean.info()
tickets_clean.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ticket_id                    2000 non-null   object 
 1   account_id                   2000 non-null   object 
 2   submitted_at                 2000 non-null   object 
 3   closed_at                    2000 non-null   object 
 4   resolution_time_hours        2000 non-null   float64
 5   priority                     2000 non-null   object 
 6   first_response_time_minutes  2000 non-null   int64  
 7   satisfaction_score           1175 non-null   float64
 8   escalation_flag              2000 non-null   bool   
dtypes: bool(1), float64(2), int64(1), object(5)
memory usage: 127.1+ KB


,ticket_id,account_id,submitted_at,closed_at,resolution_time_hours,priority,first_response_time_minutes,satisfaction_score,escalation_flag
0,T-0024de,A-712f1c,2023-07-27,2023-07-28 03:00:00,27.0,high,74,NaN,False
1,T-4d04b9,A-e43bf7,2024-07-08,2024-07-09 03:00:00,27.0,urgent,144,NaN,False
2,T-d5e12f,A-0f3e88,2024-10-17,2024-10-17 19:00:00,19.0,urgent,93,4.0,False
3,T-dfce9a,A-4c56c9,2024-09-08,2024-09-09 23:00:00,47.0,medium,126,5.0,False
4,T-c59f77,A-6f8ad2,2024-11-30,2024-12-01 02:00:00,26.0,medium,8,NaN,False


In [14]:
tickets_clean.to_csv("data_cleaned/dim_support_tickets_clean.csv", index=False)
print(" Saved data_cleaned/dim_support_tickets_clean.csv")
print(f"   {tickets_clean.shape[0]} rows, {tickets_clean.shape[1]} columns")


 Saved data_cleaned/dim_support_tickets_clean.csv
   2000 rows, 9 columns
